In [2]:
%load_ext autoreload
%autoreload 2 

* 이미지를 VAE로 Embedding하여 얻은 vector의 similarity와 풍속데이터의 Similarity 비교
* 체크해야 할 것
    - 풍속기록이 있는 데이터만 필요하며, 일부는 Train 일부는 test 이를 구분해야 할 필요가 있음

In [3]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

import yaml
import torch
from torchvision import transforms, datasets
from torch.utils.data import SubsetRandomSampler, DataLoader, TensorDataset

from src.hillshpaeDataModule import MySimCLRTrainDataTransform
from simCLR import *
from src import *

In [4]:
# Wind data load

with open('../../Share_Data/windspeed/_Main_dataset.pickle', 'rb') as f:
    windspeed_dataset = pickle.load(f)

# --- filter station for WAS journal!!
with open('../../Share_Data/windspeed/WAS_except_station_list_under_30_m.pickle', 'rb') as f:
    except_list = pickle.load(f)

new_windspeed_dataset = {}
for key in windspeed_dataset:
    if key not in except_list:
        new_windspeed_dataset[key] = windspeed_dataset[key]
        
windspeed_dataset = new_windspeed_dataset
# -------------------------------------- #
    
station_list = list(windspeed_dataset.keys())
windspeed_list = []
for item in windspeed_dataset.values():
    windspeed_list.append(item[3])
windspeed_list = np.array(windspeed_list)

## Analysis
---

[Experiment_1](#Experiment_1) - 위치기반 SVR

### Experiment_1 
* 위치기반하여 SVR
* k-hold-vaildation

In [6]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [7]:
# 거리로만
from IPython.display import clear_output

X = np.array(list(windspeed_dataset.values()))[:,:2]
windspeed_list = windspeed_list.reshape(-1)

clear_output()

In [12]:
kf = KFold(n_splits=10)
kf.get_n_splits(X)

10

In [13]:
result_list =[]
ans_list = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = windspeed_list[train_index], windspeed_list[test_index]
    
    regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
    regr.fit(X_train, Y_train)
    result_list.append(regr.predict(X_test))
    ans_list.append(Y_test)

In [14]:
error_list = list(map(lambda x : result_list[x]-ans_list[x], np.arange(kf.n_splits)))
total_error = list(map(lambda x : np.mean(np.abs(x)), error_list))

In [16]:
print('mean of error : {}'.format(np.mean(total_error)))

mean of error : 3.301263627858107


* 생각보다 잘 나오긴하는데
* 해당 Task는 굳이 NN을 사용하지 않아도 되지

In [17]:
# Hyperparameter

lr = 5e-4
epochs = 5000

kf = KFold(n_splits=10)
normal_x = StandardScaler().fit_transform(X)
log_y = np.log(windspeed_list)

In [18]:
nn_list = []
nn_ans_list = []

#loss function
loss_fn = torch.nn.MSELoss(reduction='mean')

for train_index, test_index in kf.split(normal_x):
    X_train = torch.tensor(normal_x[train_index], dtype=torch.float32)
    X_test = torch.tensor(normal_x[test_index], dtype=torch.float32)
    Y_train = torch.tensor(windspeed_list[train_index].reshape(-1,1), dtype=torch.float32)
    Y_test = torch.tensor(windspeed_list[test_index].reshape(-1,1), dtype=torch.float32)
    
    #model reset
    model = torch.nn.Sequential(
            torch.nn.Linear(2, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32,32),
            torch.nn.ReLU(),
            torch.nn.Linear(32,1))
    
    #Opt
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    schedulr = torch.optim.lr_scheduler.ReduceLROnPlateau(opt)
    
    for epoch in np.arange(epochs):
        y_pred = model(X_train)
        loss = loss_fn(y_pred, Y_train)
        opt.zero_grad()
        loss.backward()
        opt.step()
        schedulr.step(loss)
        if epoch % 100 == 0:
            print('step : {}, loss : {}'.format(epoch, loss.item()/len(y_pred)))
    
    print('---- training end ----')
    y_pred = model(X_test)
    loss = loss_fn(y_pred, Y_test)
    print('val loss : {}'.format(loss.item()))
    nn_list.append(y_pred.detach().numpy())
    nn_ans_list.append(Y_test.detach().numpy())
    clear_output()

In [19]:
result = list(map(lambda x : np.mean(np.abs(nn_ans_list[x] - nn_list[x])), np.arange(len(nn_ans_list))))
print(result)
print(np.mean(result))

[3.7572541, 3.344172, 2.216308, 3.5453944, 3.5181408, 2.822469, 2.7147949, 4.159629, 4.54025, 3.9485848]
3.4566998


In [149]:
result = list(map(lambda x : np.mean(np.abs(nn_ans_list[x] - nn_list[x])), np.arange(len(nn_ans_list))))
print(result)
print(np.mean(result))

[3.3729942, 3.5524478, 2.3723252, 2.6969068, 2.8846362, 3.3595045, 2.7867486, 4.1057005, 4.5895967, 4.1195188]
3.3840377


In [142]:
result = list(map(lambda x : np.mean(np.abs(nn_ans_list[x] - nn_list[x])), np.arange(len(nn_ans_list))))
print(result)
print(np.mean(result))

[3.47251, 3.5159757, 2.325559, 2.9339614, 3.0844433, 3.176426, 2.8345044, 4.050174, 4.5520773, 4.229054]
3.4174685


### NN 

* with latent vector

In [9]:
latent_hill = np.load('data/WAS_hill_latent.npy')
latent_rgb = np.load('data/WAS_RGB_latent.npy')
latent_cart = np.load('data/WAS_CART_latent.npy')

latent_all = np.concatenate([latent_rgb, latent_hill, latent_hill], axis=-1)

In [11]:
# SVR
# Hyperparameter

kf = KFold(n_splits=10)
#latent = model_projection/model_n
normal_x = np.concatenate([StandardScaler().fit_transform(X), latent_all], axis=-1)
log_y = np.log(windspeed_list)

In [12]:
kf = KFold(n_splits=10)
kf.get_n_splits(normal_x)

10

In [13]:
result_list =[]
ans_list = []

for train_index, test_index in kf.split(X):
    X_train, X_test = normal_x[train_index], normal_x[test_index]
    Y_train, Y_test = windspeed_list[train_index], windspeed_list[test_index]
    
    #regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
    reg = SVR(C=1.0, epsilon=0.2)
    reg.fit(X_train, Y_train)
    result_list.append(reg.predict(X_test))
    ans_list.append(Y_test)

In [14]:
error_list = list(map(lambda x : result_list[x]-ans_list[x], np.arange(kf.n_splits)))
total_error = list(map(lambda x : np.mean(np.abs(x)), error_list))

In [15]:
print('mean of error : {}'.format(np.mean(total_error)))

mean of error : 3.2148897180598617


#### NN

In [17]:
# Hyperparameter

lr = 1e-4
epochs = 2000

In [ ]:
train_loss = []
val_loss = []
total_result = []
nn_list = []
nn_ans_list = []

#loss function
loss_fn = torch.nn.MSELoss(reduction='mean')

for train_index, test_index in kf.split(normal_x):
    X_train = torch.tensor(normal_x[train_index], dtype=torch.float32)
    X_test = torch.tensor(normal_x[test_index], dtype=torch.float32)
#    Y_train = torch.tensor(log_y[train_index].reshape(-1,1), dtype=torch.float32)
#    Y_test = torch.tensor(log_y[test_index].reshape(-1,1), dtype=torch.float32)
    Y_train = torch.tensor(windspeed_list[train_index].reshape(-1,1), dtype=torch.float32)
    Y_test = torch.tensor(windspeed_list[test_index].reshape(-1,1), dtype=torch.float32)



    #model reset
    model = torch.nn.Sequential(
            torch.nn.Linear(2+32*3, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024,1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024,1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024,512),
            torch.nn.ReLU(),
            torch.nn.Linear(512,1))

    #Opt
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    schedulr = torch.optim.lr_scheduler.ReduceLROnPlateau(opt)
    
    temp_train_loss = []
    temp_test_loss = []
    
    for epoch in np.arange(epochs):
        y_pred = model(X_train)
        loss = loss_fn(y_pred, Y_train)
        temp_train_loss.append(float(loss.data))
        opt.zero_grad()
        loss.backward()
        opt.step()
        schedulr.step(loss)
        
        y_pred = model(X_test)
        loss = loss_fn(y_pred, Y_test)
        temp_test_loss.append(float(loss.data))
        
    train_loss.append(temp_train_loss)
    val_loss.append(temp_test_loss)
    
#    nn_list.append(np.exp(y_pred.detach().numpy()))
#    nn_ans_list.append(np.exp(Y_test.detach().numpy()))
    nn_list.append(y_pred.detach().numpy())
    nn_ans_list.append(Y_test.detach().numpy())
    total_result.append(np.mean(list(map(lambda x : np.mean(np.abs(nn_ans_list[x] - nn_list[x])), np.arange(len(nn_ans_list))))))

In [ ]:
for i in np.arange(len(train_loss)):
#    plt.plot(np.arange(len(train_loss[i])), train_loss[i])
    plt.plot(np.arange(len(val_loss[i]))[50:], val_loss[i][50:])

In [ ]:
print(total_result)
print(np.mean(total_result))

In [173]:
torch.save(model.state_dict(), 'WAS_model')

In [174]:
test_model = torch.nn.Sequential(
            torch.nn.Linear(34, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024,1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024,1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024,512),
            torch.nn.ReLU(),
            torch.nn.Linear(512,1))

In [176]:
test_model.load_state_dict(torch.load('WAS_model'))

<All keys matched successfully>

In [114]:
print(total_result)
print(np.mean(total_result))

[3.312671, 3.347396, 3.160229, 3.1332514, 3.0017517, 3.1042824, 3.0894096, 3.1937537, 3.4063134, 3.4999619]
3.2249017


* 올....